In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [14]:
import os
import pandas as pd
import transformers

#### helper function below

In [15]:
def zeroshot(model_name, csv_name):
  train_path = "drive/MyDrive/fyp/data_ready/all_data.csv"
  # train_path = "../data_ready/all_data.csv"
  print(f"zero_shot data exist? : {os.path.exists(train_path)}")

  data=pd.read_csv(train_path).sample(frac=1)#.reset_index() #shuffle dataframe

  data_text, data_lables = data['text'], data['class']
  possible_data_labels = ['exclusive','inclusive']

  classifier = transformers.pipeline("zero-shot-classification", model=model_name)
  zero_shot_results = classifier(list(data_text), possible_data_labels)

  text = list()
  prediction = list()

  for result in zero_shot_results:
    text.append(result.get("sequence"))
    inc = result.get('scores')[0]
    exc = result.get('scores')[1]
    if inc>exc:
      prediction.append(1)
    else:
      prediction.append(0)

  #### ADD EXTRA RESULTS #####
  extra_example = classifier("I HATE YOU!", possible_data_labels)
  text.append("I HATE YOU!")
  # data_lables.append(0)
  inc = extra_example.get('scores')[0]
  exc = extra_example.get('scores')[1]
  if inc>exc:
    prediction.append(1)
  else:
    prediction.append(0)
  #### END EXTRA SENTENCES TO CLASSIFY ####

  results_formatted = pd.DataFrame()
  results_formatted['text'] = text
  results_formatted['class'] = data_lables
  results_formatted['predicted'] = prediction

  results_formatted.to_csv("drive/MyDrive/fyp/predictions/"+csv_name+".csv", index=False)
  # results_formatted.to_csv("../predictions/"+csv_name+".csv", index=False)


### Run all zeroshot classifiers

In [16]:
zeroshot("morit/french_xlm_xnli","ZEROSHOT_french_xlm_xnli")

zero_shot data exist? : True


Ignoring args : (['exclusive', 'inclusive'],)


TypeError: 'NoneType' object is not subscriptable